# PIB Filtering and Trend-Cycle Decomposition
---

This notebook scans the ../data/raw/ directory for the **latest available GDP series** from IBGE (series can be downloaded using IBGE.ipynb notebook).

It automatically detects CSV files with filenames following the standard naming pattern, and for each unique series (identified by table and variable), it keeps only the most recent file.

These files are matched with IBGE metadata to present the user with an intuitive interface to select:

- Real or Nominal GDP  
- Seasonally Adjusted or Non-Adjusted  
- Quarterly or Annual Frequency  

Once a series is selected, the notebook loads and processes the data. The following transformations and filters are applied:

- **Natural Log** — computed using **NumPy**
- **First Difference** — computed using **pandas**
- **Percentage Change** — computed using **pandas**
- **Hodrick-Prescott Filter** — using **statsmodels**
- **Baxter-King Filter** — using **statsmodels**
- **Christiano-Fitzgerald Filter** — using **statsmodels**

All of these operations are implemented using well-established, trusted Python libraries for time series and econometric analysis:

Finally, the notebook provides an interactive plotting interface so you can visually explore trends, cycles, and transformations of the GDP series with ease.

This environment is ideal for filtering, comparing smoothing methods, and preparing data for macroeconomic analysis and visualization.


## Notebook Setup and Dependencies Loading
---

Run the cell below in order to load dependencies, metadata, and start the logging session.

In [1]:
# Importing external libraries and functions
import os
import re
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import ipywidgets as widgets
import matplotlib.pyplot as plt

from datetime import datetime
from IPython.display import display, Markdown, clear_output

# Add the 'src' folder to the Python path so project-specific modules can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

# Import project-specific functions
from logger import start_logger
from ibge import load_ibge_series_metadata
from utils import compute_file_hash
from ui import file_explorer, raw_cleanup_widget

# Enable automatic reloading of modules when their source code changes
%reload_ext autoreload
%autoreload 2

# Define Session ID
session_type = "Filtering"
session_ID = datetime.now().strftime("%Y%m%d_%H%M%S")

# Setup Logging
log_file_name = f"../logs/{session_type}_{session_ID}.log"
logger_name = "root"
logger = start_logger(logger_name, log_file_name)

raw_cleanup_widget()

2025-03-31 21:03:41,342 - INFO - Logger started. File path: ../logs/Filtering_20250331_210341.log


## Select GDP data from available series
---

In [2]:
# Load metadata
df_ibge_series_metadata = load_ibge_series_metadata()
GDP_file_explorer_refs = file_explorer(df_ibge_series_metadata )

2025-03-31 21:03:43,148 - INFO - Loaded IBGE Metadata from file: ../data/metadata/ibge_series.json


## Filter data
---

In [7]:
# Function to get data from Widget selection
def get_data(selected_filename): 
    # Load the DataFrame
    df = pd.read_csv(selected_filename)

    # Convert columns if present
    if "data" in df.columns:
        df["data"] = pd.to_datetime(df["data"], errors="coerce")

    if "valor" in df.columns:
        df["valor"] = pd.to_numeric(df["valor"], errors="coerce")
    return df

# Get data acoridng to Widget Selection
df = get_data(GDP_file_explorer_refs["get_selected_file"]())

# Create log of valor
df["log_valor"] = df["valor"].apply(lambda x: np.log(x) if x > 0 else np.nan)

# Create first difference of log_valor
df["fdiff_cycle"] = df["log_valor"].diff()
df["fdiff_trend"] = df["log_valor"] - df["fdiff_cycle"]

# Create % change of log_valor
df["pct_change_cycle"] = df["valor"].pct_change()
df["pct_change_trend"] = df["log_valor"] - df["pct_change_cycle"]

# HP Filter
df["hp_cycle"], df["hp_trend"]  = sm.tsa.filters.hpfilter(df["log_valor"], 1600)

# BK Filter
df["bk_cycle"] = sm.tsa.filters.bkfilter(df["log_valor"], 6, 32, 12)
df["bk_trend"] = df["log_valor"] - df["bk_cycle"] 

# CF Filter
df["ck_cycle"], df["ck_trend"]  = sm.tsa.filters.cffilter(df["log_valor"], 6,32,False)

# OLS Regression
#--------------------------
# Independent variable (x): time
x = df['data'].apply(lambda d: d.toordinal())
x = sm.add_constant(x)  # Adds intercept term

# Dependent variable (y): value
y = df['log_valor']

# Fit model
model = sm.OLS(y, x).fit()

# Get all coefficients
coefficients = model.params

# Add predicted values (trend) to the DataFrame
df['OLS_trend'] = model.predict(x)

# Calculate the cycle (residual)
df['OLS_cycle'] = df['log_valor'] - df['OLS_trend']

# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_valor   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                     1014.
Date:                Mon, 31 Mar 2025   Prob (F-statistic):           1.45e-58
Time:                        21:06:25   Log-Likelihood:                 155.99
No. Observations:                 116   AIC:                            -308.0
Df Residuals:                     114   BIC:                            -302.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -40.1654      1.418    -28.332      0.0

### Plot filtered GDP data
---

In [8]:
# Create the widget to select columns (except 'data')
column_selector = widgets.SelectMultiple(
    options=[col for col in df.columns if col != "data"],
    description="Y Columns:",
    layout=widgets.Layout(width="400px", height="200px")
)

# Output area for the plot
plot_output = widgets.Output()

# Function to update the plot
def update_plot(change):
    with plot_output:
        clear_output()
        selected = list(column_selector.value)

        if not selected:
            print("Select at least one column to plot.")
            return

        # Plot
        sns.set_theme()
        sns.set_context("notebook")
        plt.figure(figsize=(12, 6))

        for col in selected:
            plt.plot(df["data"], df[col], label=col)

        plt.xlabel("Date")
        plt.ylabel("Value")
        plt.title("Selected Columns Over Time")
        plt.legend()
        sns.despine()
        plt.tight_layout()
        plt.show()

# Connect widget to function
column_selector.observe(update_plot, names="value")

# Display UI
display(widgets.HTML("<b>Select columns to plot (X axis is always 'data'):</b> Use CTRL or CMD to select multiple rows"))
display(
    widgets.HBox([
        column_selector,
        plot_output
    ])
)

# Initial plot
update_plot({"new": column_selector.value})


HTML(value="<b>Select columns to plot (X axis is always 'data'):</b> Use CTRL or CMD to select multiple rows")

## Select Inflation Data
---

In [ ]:
# Load metadata
IPCA_file_explorer_refs = file_explorer(df_ibge_series_metadata )

In [33]:
# Load selected IPCA file from the file explorer widget
dfa = get_data(IPCA_file_explorer_refs["get_selected_file"]())

# Convert monthly percent change to decimal (for compounding)
dfa["decimal"] = 1 + dfa["valor"] / 100

# Set date as index and resample to quarterly using compounded product
dfa.set_index("data", inplace=True)
dfa = dfa.resample("QE").prod()  # 'QE' = quarter end

# Shift quarterly dates from end-of-quarter to start-of-quarter
dfa = dfa.reset_index()[["data", "decimal"]]
dfa["data"] = dfa["data"] + pd.Timedelta(days=1)

# Convert decimal back to percent change and drop intermediate column
dfa["pi"] = (dfa["decimal"] - 1) * 100
dfa = dfa[["data", "pi"]]

## Compare Forecast Errors
---

In [ ]:
# Get data acoridng to Widget Selection
df = get_data()

# Set Window Size (i.e. 4*10 = 40 quarters = 10 years of quarterly data)
ws = 4*10

# Set Forecast Size (i.e. 4 = 4 quarters of forecast)
fs = 4

# Calculate number of windows in set
nw = len(df)-ws-fs

In [71]:
# Window Counter, from 0 to nw
i = 0

# Get Window Data
dfa = df[i:i+ws]

# Get Data to be forecasted
dfx = df[i+ws:i+ws+fs]

In [ ]:
# Independent variable (x): time
x = dfa['data'].apply(lambda d: d.toordinal())
x = sm.add_constant(x)  # Adds intercept term

# Dependent variable (y): value
y = dfa['log_valor']

# Fit model
model = sm.OLS(y, x).fit()

# Get all coefficients
coefficients = model.params

# Add predicted values (trend) to the DataFrame
df['OLS_trend'] = model.predict(x)

# Calculate the cycle (residual)
df['OLS_cycle'] = df['log_valor'] - df['OLS_trend']

## Debug RAW Files
---

In [67]:
import os
import re
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# ─────────────────────────────────────
# STEP 1: Scan folder and extract file info
raw_data_dir = "../data/raw/"

# Matches: IBGE_20250329_151203_T6784-V9808_153017.csv
filename_pattern = re.compile(
    r"(?P<source>IBGE|BCB)_(?P<session>\d{8}_\d{6})_T(?P<table>\d+)-V(?P<variable>\d+)_(?P<filetime>\d{6})\.csv"
)

records = []

for fname in os.listdir(raw_data_dir):
    match = filename_pattern.match(fname)
    if match:
        rec = match.groupdict()
        rec["filename"] = os.path.join(raw_data_dir, fname)
        records.append(rec)

df_all_files = pd.DataFrame(records)

# Add combined column for selection
df_all_files["T-V"] = "T" + df_all_files["table"] + "-V" + df_all_files["variable"]

# ─────────────────────────────────────
# STEP 2: Build UI widgets

# Unique source list
source_selector = widgets.Dropdown(
    options=sorted(df_all_files["source"].unique()),
    value="IBGE",
    description="Source:"
)

# Will be filled dynamically
tv_selector = widgets.Dropdown(
    options=[],
    description="Table-Var:"
)

series_name_label = widgets.HTML("<b>Series Name:</b> (will update on selection)")


# Output area to display results
result_output = widgets.Output()

# ─────────────────────────────────────
# STEP 3: UI logic to update dropdowns and show table

def update_tv_options(change):
    selected_source = change["new"]
    filtered = df_all_files[df_all_files["source"] == selected_source]
    tv_selector.options = sorted(filtered["T-V"].unique())
    if tv_selector.options:
        tv_selector.value = tv_selector.options[0]

# Container for dropdown and plot
file_selector_container = widgets.VBox()
file_plot_output = widgets.Output()
filename_label = widgets.HTML("<b>Selected file:</b> (will appear here)")
def show_files_table(change):

# Try to look up the name using table and variable


    with result_output:
        clear_output()
        file_selector_container.children = []  # Reset

        source = source_selector.value
        tv = tv_selector.value
        filtered = df_all_files[
            (df_all_files["source"] == source) & 
            (df_all_files["T-V"] == tv)
        ].copy()
        
        try:
            meta_row = df_ibge_series_metadata[
                (df_ibge_series_metadata["table"] == int(tv.split("-")[0][1:])) &
                (df_ibge_series_metadata["variable"] == int(tv.split("-")[1][1:]))
            ].iloc[0]
            series_name_label.value = f"Series Name: {meta_row.name}"
        except:
            series_name_label.value = "Series Name: (not found)"

        if filtered.empty:
            display(widgets.HTML("No files found for selected source and series."))
            return

        # Compute hash
        filtered["file_hash"] = filtered["filename"].apply(compute_file_hash)
        def get_valor_bounds(filepath):
            try:
                df = pd.read_csv(filepath, usecols=["valor"])
                df["valor"] = pd.to_numeric(df["valor"], errors="coerce")
                df = df.dropna()
                if df.empty:
                    return (np.nan, np.nan)
                return (df.iloc[0]["valor"], df.iloc[-1]["valor"])
            except Exception as e:
                print(f"Error reading {filepath}: {e}")
                return (np.nan, np.nan)

        # Apply to each file
        filtered[["valor_first", "valor_last"]] = filtered["filename"].apply(
            lambda path: pd.Series(get_valor_bounds(path))
        )
        # Generate label for each option
        filtered["label"] = filtered.apply(
            lambda row: f"{row['session']} - {row['filetime']} - "
                        f"{row['file_hash'][:4]}...{row['file_hash'][-4:]} | "
                        f"valor: {row['valor_first']:.2f} → {row['valor_last']:.2f}",
            axis=1
        )
        # Map labels to filenames
        filtered = filtered.sort_values(by=["session", "filetime"], ascending=False)
        file_map = dict(zip(filtered["label"], filtered["filename"]))

        # Create dropdown
        file_dropdown = widgets.Select(
            options=list(file_map.keys()),
            description="File:",
            layout=widgets.Layout(width="700px", height="150px")
        )

        # Define plot function
        def plot_selected_file(change):
            with file_plot_output:
                clear_output()
                selected_label = change["new"]
                file_path = file_map[selected_label]
                filename_label.value = f"Selected file: {file_path}"

                try:
                    df = pd.read_csv(file_path)
                    if "data" in df.columns:
                        df["data"] = pd.to_datetime(df["data"], errors="coerce")
                    if "valor" in df.columns:
                        df["valor"] = pd.to_numeric(df["valor"], errors="coerce")

                        # Plot
                        sns.set_theme()
                        sns.set_context("notebook")
                        plt.figure(figsize=(12, 6))
                        sns.lineplot(data=df, x="data", y="valor")
                        plt.xlabel("Date")
                        plt.ylabel("Value")
                        plt.title(f"{source} {tv} - {selected_label}")
                        sns.despine()
                        plt.tight_layout()
                        plt.show()
                    else:
                        print("Column 'valor' not found.")
                except Exception as e:
                    print(f"Error reading or plotting file: {e}")

        # Link change event
        file_dropdown.observe(plot_selected_file, names="value")

        # Trigger first plot
        plot_selected_file({"new": file_dropdown.value})

        # Add widgets to UI
        file_selector_container.children = [file_dropdown, file_plot_output]
        display(file_selector_container)
        
# Attach logic
source_selector.observe(update_tv_options, names="value")
tv_selector.observe(show_files_table, names="value")

# Initialize dropdowns
update_tv_options({"new": source_selector.value})
show_files_table(None)

# ─────────────────────────────────────
# Display full UI
display(widgets.VBox([
    widgets.HTML("<h3>🔎 Analyze All Files by Source and Series</h3>"),
    source_selector,
    tv_selector,
    series_name_label,   
    filename_label, 
    result_output,

]))


In [34]:
tv = "T6612-V9318"
meta_row = df_ibge_series_metadata[
    (df_ibge_series_metadata["table"] == int(tv.split("-")[0][1:])) &
    (df_ibge_series_metadata["variable"] == int(tv.split("-")[1][1:]))
].iloc[0]

In [46]:
meta_row.name

'GDP Quarterly - 1995 Prices 6612-9318-90707'

In [24]:
df_ibge_series_metadata["variable"]

name
GDP Quarterly - Real - 1620-583-90707                                   583
GDP Quarterly - Real (Seasonally Adjusted) 1621-584-90707               584
GDP Quarterly - Current Prices 1846-585-90707                           585
GDP Quarterly - 1995 Prices 6612-9318-90707                            9318
GDP Quarterly - 1995 Prices (Seasonally Adjusted) 6613-9319-90707      9319
GDP Annual - Current Prices 6784-9808                                  9808
GDP Annual - Previous Year Prices 6784-9809                            9809
IPCA Montly Variation (Seasonally Adjusted) - From Jan/1998 118-306     306
Name: variable, dtype: int64

In [33]:
int(tv.split("-")[0][1:])

6612

In [ ]:
# Load metadata
df_ibge_series_metadata = load_ibge_series_metadata()

# Ensure index is name
df_ibge_series_metadata.index.name = "name"
df_ibge_series_metadata = df_ibge_series_metadata.reset_index()

# ───────────────────────────────────────────────
# Step 1: Scan directory and find latest CSV for each (table, variable)

raw_data_dir = "../data/raw/"
filename_pattern = re.compile(
    r"IBGE_\d{8}_\d{6}_T(?P<table>\d+)-V(?P<variable>\d+)_(?P<timestamp>\d{6})\.csv"
)

latest_files = {}

for fname in os.listdir(raw_data_dir):
    match = filename_pattern.match(fname)
    if match:
        table = int(match.group("table"))
        variable = int(match.group("variable"))
        timestamp = match.group("timestamp")
        key = (table, variable)
        if key not in latest_files or timestamp > latest_files[key]["timestamp"]:
            latest_files[key] = {
                "filename": os.path.join(raw_data_dir, fname),
                "timestamp": timestamp
            }

# ───────────────────────────────────────────────
# Step 2: Join metadata with file info

# Convert to DataFrame for easy display
df_files = pd.DataFrame([
    {"table": k[0], "variable": k[1], "filename": v["filename"], "timestamp": v["timestamp"]}
    for k, v in latest_files.items()
])

# Merge with metadata
df_merged = pd.merge(
    df_ibge_series_metadata,
    df_files,
    on=["table", "variable"],
    how="inner"
).set_index("name")

# ───────────────────────────────────────────────
# Step 3: Create widget to display and select a file

# Dropdown with series names
series_dropdown = widgets.Dropdown(
    options=df_merged.index.tolist(),
    description="Series:",
    layout=widgets.Layout(width='500px')
)

# Output area for metadata preview
file_preview_out = widgets.Output()

def update_preview(change):
    with file_preview_out:
        clear_output()
        selected = change["new"]
        row = df_merged.loc[selected]

        # Display metadata
        display(Markdown(f"**Selected file:** `{row['filename']}`"))
        display(Markdown(f"**Table:** {row['table']} &nbsp;&nbsp; **Variable:** {row['variable']}"))
        display(Markdown(f"**Frequency:** {row['frequency']}  &nbsp;&nbsp; **Category:** {row['category']}"))

        # Try to load and plot data
        try:
            df = pd.read_csv(row['filename'])

            # Basic checks or parsing
            if "data" in df.columns:
                df["data"] = pd.to_datetime(df["data"], errors="coerce")
            if "valor" in df.columns:
                df["valor"] = pd.to_numeric(df["valor"], errors="coerce")

            # Plot
            sns.set_theme()
            sns.set_context("notebook")
            plt.figure(figsize=(12, 6))
            sns.lineplot(data=df, x="data", y="valor")
            plt.xlabel("Date")
            plt.ylabel("Value")
            plt.title(f"IBGE Series: {selected}")
            sns.despine()
            plt.tight_layout()
            plt.show()
        except Exception as e:
            display(Markdown(f"**Error loading or plotting data:** {e}"))

# Initial display
update_preview({"new": series_dropdown.value})
series_dropdown.observe(update_preview, names="value")

# Display
display(Markdown("### 📁 Select one of the available IBGE series from disk"))
display(series_dropdown, file_preview_out)

# ➕ You can later get the selected file like this:
# selected_filename = df_merged.loc[series_dropdown.value, "filename"]
